# 第４回小テスト(全3問)
## 第１問  SVM

sklearn.datasetsに収録されているBreast Cancer Wisconsin (Diagnostic) Data Setという乳がんの検査のために集められたデータセットをSVMで分析した上でindex=100の予測ラベルと正解ラベルを出力しなさい。SVMを使う際にはパラメータのチューニングも行うこと。

In [1]:
# 必要なライブラリの読み込み
import numpy as np
import pandas as pd

# brest_cancerデータセットの読み込み
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()

# データをDataFrameに変換
df_data = pd.DataFrame(bc.data, columns=bc.feature_names)
df_target = pd.DataFrame(bc.target, columns=['class'])
df = pd.concat([df_data, df_target], axis=1)

# 説明変数と目的変数は変えなくて良い
X = df.loc[:, ['worst perimeter', 'mean concave points']].values
y = df.loc[:, ['class']].values

# scikit-learnの仕様に合わせて､一列のベクトルに変換
y = y.reshape(-1)

### 1)
データを標準化してください。

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.fit_transform(X)

### 2)
データセットを分割してください。

In [3]:
from sklearn.model_selection import train_test_split
X_std_train,X_std_test,y_train,y_test=train_test_split(X_std,y,test_size=0.3,random_state=0)

### 3)
グリッドサーチを利用してパラメータをチューニングし、最適なパラメータを表示してください。  
ただし層化交差検証法を用いること。  
またその時のtrainデータをtestデータの正答率を表示してください。  
ただしパラメータの候補としては以下のものを使うこと。  
C　　0.1, 0.3, 0.5, 1.0  
gamma    0.01, 0.03, 0.05, 0.07

In [6]:
# インスタンスの生成
# パラメータのチューニング
param_grid = {'C': [0.1, 0.3, 0.5, 1.0],
              'gamma': [0.01, 0.03, 0.05, 0.07]}
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
gs_svc = GridSearchCV(SVC(), param_grid, cv=skf) 
gs_svc.fit(X_std_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gamma': [0.01, 0.03, 0.05, 0.07], 'C': [0.1, 0.3, 0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [7]:
#求められた最適なパラメータの表示
gs_svc.best_params_

{'C': 0.5, 'gamma': 0.03}

In [9]:
# trainデータとtestデータの正答率を出力
print('train:%.3f'%gs_svc.score(X_std_train,y_train))
print('test:%.3f'%gs_svc.score(X_std_test,y_test))

train:0.935
test:0.930


### 3)
indexが100のサンプルの正解と予測のクラスラベルを出力してください。

In [21]:
index =100
print('predict:%d'%gs_svc.predict(X_std[index].reshape(1,-1)))
print('answer:%d'%y[index])

predict:1
answer:0


## 第２問  パーセプトロン・ロジスティック回帰

パーセプトロンとロジスティック回帰のそれぞれどのようにして分類を行うか、またいかにして学習するか説明しなさい。
またそれぞれ線形分離が不可能な問題に対して使用可能かどうか述べなさい。

In [ ]:
パーセプトロン
入力の線形結合を活性化関数により変換し、その出力が+1か-1かで分類する。
(正解か不正解か)

予測の間違いの個数を減らすように学習する。
線形分離不可能な場合は不正解数が収束しないため使用できない。

In [ ]:
ロジスティック回帰
入力の線形結合をロジスティック関数に通して0~1の範囲に収める。その後0.5を境界にしてクラスラベルを判別する。出力は予測確率を意味している。
(どれだけ確信を持てたか)

パラメータの推定には最尤推定を用いる。与えられているデータに対する予測がもっとも上手く行くようなパラメータになる。
線形分離不可能な問題には使えない。 --> uが線形結合を仮定しないと使えないから？　教えて井出きゅん！

## 第３問  欠損値処理

pandasのDataFrameを利用して､以下のサンプルデータに対して欠損値処理やエンコーディングを行いなさい。

In [22]:
# サンプルとなるデータセットの作成
import numpy as np
from numpy import nan 
import pandas as pd

In [23]:
# 商品の購買履歴データのサンプルを生成
df = pd.DataFrame({'DATE':  [20170801, 20170801, 20170802, 20170802, 20170803, 20170803, 20170803, 20170805, 20170805, 20170805],
                   'NECK':  [41, nan, 38, 46, nan, 49, nan, nan, nan, 42],
                   'BODY':  [84, 92, 69, 90, nan, 64, nan, 74, nan, 86],
                   'SIZE':  ['S', 'XL', "S", 'XL', 'M', 'M', 'M', 'M', 'M', 'XL'],
                   'COLOR': ['BL', 'RD', 'Y', 'GR', 'GR', 'RD', 'BL', 'Y', 'BL', 'GR'],
                   'class': ['A', 'C', 'B', 'B', 'C', 'A', 'A', 'A', 'C', 'C']},
                 columns=['DATE', 'NECK', 'BODY', 'SIZE', 'COLOR', 'class'])
df

,DATE,NECK,BODY,SIZE,COLOR,class
0,20170801,41.0,84.0,S,BL,A
1,20170801,NaN,92.0,XL,RD,C
2,20170802,38.0,69.0,S,Y,B
3,20170802,46.0,90.0,XL,GR,B
4,20170803,NaN,NaN,M,GR,C
5,20170803,49.0,64.0,M,RD,A
6,20170803,NaN,NaN,M,BL,A
7,20170805,NaN,74.0,M,Y,A
8,20170805,NaN,NaN,M,BL,C
9,20170805,42.0,86.0,XL,GR,C


### 1)
NECKに欠損があるサンプルのみ削除してください。

In [27]:
drop_neck = df.dropna(subset=['NECK'])
drop_neck

,DATE,NECK,BODY,SIZE,COLOR,class
0,20170801,41.0,84.0,1,BL,A
2,20170802,38.0,69.0,1,Y,B
3,20170802,46.0,90.0,4,GR,B
5,20170803,49.0,64.0,2,RD,A
9,20170805,42.0,86.0,4,GR,C


### 2)
シャツのサイズと整数を対応させる辞書を作成し、それをもとにマッピングを実行してください。

In [25]:
size_mapping = {'S':1, 'M':2, 'L':3, 'XL':4}

In [26]:
df.SIZE = df.SIZE.map(size_mapping)
df

,DATE,NECK,BODY,SIZE,COLOR,class
0,20170801,41.0,84.0,1,BL,A
1,20170801,NaN,92.0,4,RD,C
2,20170802,38.0,69.0,1,Y,B
3,20170802,46.0,90.0,4,GR,B
4,20170803,NaN,NaN,2,GR,C
5,20170803,49.0,64.0,2,RD,A
6,20170803,NaN,NaN,2,BL,A
7,20170805,NaN,74.0,2,Y,A
8,20170805,NaN,NaN,2,BL,C
9,20170805,42.0,86.0,4,GR,C
